[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Vampire-Chan/SDUI/blob/main/ComfyUI.ipynb)


In [ ]:
#@title **ComfyUI Environment Auto-Setup with Update Support**
import os
from IPython.display import clear_output

# ✅ User-configurable options
USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}

# ✅ Define workspace path
if USE_GOOGLE_DRIVE:
    print("🔗 Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
else:
    WORKSPACE = "./ComfyUI"

COMFY_REPO = "https://github.com/comfyanonymous/ComfyUI"
MANAGER_REPO = "https://github.com/ltdrdata/ComfyUI-Manager.git"
CUSTOM_NODES_DIR = os.path.join(WORKSPACE, "custom_nodes", "ComfyUI-Manager")

# ✅ Clone or update ComfyUI
if not os.path.exists(WORKSPACE):
    print("📂 Cloning ComfyUI for the first time...")
    os.system(f"git clone {COMFY_REPO} {WORKSPACE}")
else:
    if UPDATE_COMFY_UI:
        print("🔄 Updating ComfyUI...")
        os.chdir(WORKSPACE)
        os.system("git reset --hard")
        os.system("git pull origin master")
    else:
        print("⏭️ Skipping ComfyUI update.")

# ✅ Clone or update ComfyUI-Manager
if not os.path.exists(CUSTOM_NODES_DIR):
    print("📂 Cloning ComfyUI-Manager for the first time...")
    os.system(f"git clone {MANAGER_REPO} {CUSTOM_NODES_DIR}")
else:
    if UPDATE_COMFY_UI:
        print("🔄 Updating ComfyUI-Manager...")
        os.chdir(CUSTOM_NODES_DIR)
        os.system("git reset --hard")
        os.system("git pull origin main")
    else:
        print("⏭️ Skipping ComfyUI-Manager update.")

# ✅ Install dependencies
os.chdir(WORKSPACE)
print("📦 Installing dependencies...")
os.system(
    "pip install xformers!=0.0.18 -r requirements.txt "
    "--extra-index-url https://download.pytorch.org/whl/cu121 "
    "--extra-index-url https://download.pytorch.org/whl/cu118 "
    "--extra-index-url https://download.pytorch.org/whl/cu117"
)

print("✅ Environment setup and update complete.")
clear_output()


In [ ]:
import os, sys, time, shutil, subprocess, urllib.parse, requests, glob
from google.colab import drive

# ───────────────────────────────────────── CONFIG
MODEL_STORAGE_LOCATION = "Google Drive (Persistent)"      #@param ["Google Drive (Persistent)", "Colab Filesystem (Notebook)", "Temporary SDUI Folder (Volatile)"]
MODEL_URL  = "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/wan2.1_i2v_480p_14B_fp16.safetensors"  # default ✔️
API_KEY    = ""                                           #@param {type:"string"}
MODEL_TYPE = "checkpoints"                                #@param ["checkpoints","clip_vision","vae","loras","controlnet","style_models","upscale_models","diffusion_models","hypernetworks","gligen","custom_nodes"]

# ───────────────────────────────────────── PATHS
def setup_environment():
    if MODEL_STORAGE_LOCATION == "Google Drive (Persistent)":
        print("🔗 Mounting Google Drive…")
        drive.mount('/content/drive')
        return '/content/drive/MyDrive/ComfyUI'
    elif MODEL_STORAGE_LOCATION == "Colab Filesystem (Notebook)":
        return './ComfyUI'
    else:
        return '/content/sdui'

BASE_DIR = setup_environment()

MODEL_DIRS = {k: os.path.join(BASE_DIR, p) for k, p in {
    'checkpoints':'models/checkpoints','clip_vision':'models/clip_vision','vae':'models/vae',
    'loras':'models/loras','controlnet':'models/controlnet','style_models':'models/style_models',
    'upscale_models':'models/upscale_models','diffusion_models':'models/diffusion_models',
    'hypernetworks':'models/hypernetworks','gligen':'models/gligen','custom_nodes':'custom_nodes',
    'clip':'models/clip','configs':'models/configs'}.items()}
for p in MODEL_DIRS.values(): os.makedirs(p, exist_ok=True)

# ───────────────────────────────────────── HELPERS
def format_bar(pct, n=30):
    done = int(pct/100*n)
    return '[' + '█'*done + '-'*(n-done) + ']'

def download_with_progress(url, dest_folder, api_key=None):
    headers = {'Authorization': f'Bearer {api_key}'} if api_key else {}
    r = requests.get(url, stream=True, allow_redirects=True, headers=headers)
    r.raise_for_status()

    name = os.path.basename(urllib.parse.urlparse(url).path)
    if 'content-disposition' in r.headers:
        import cgi; _, prm = cgi.parse_header(r.headers['content-disposition'])
        name = prm.get('filename', name)

    path   = os.path.join(dest_folder, name)
    total  = int(r.headers.get('Content-Length', 0))
    MB     = 1024*1024
    print(f"📦 File size: {total/MB:.2f} MB")

    rec = 0; t0 = time.time()
    with open(path, 'wb') as f:
        for chunk in r.iter_content(8192):
            if chunk:
                f.write(chunk); rec += len(chunk)
                pct   = rec*100/total if total else 0
                speed = rec/MB/max(time.time()-t0,1)
                bar   = format_bar(pct)
                sys.stdout.write(
                    f"\r📥 {name} {bar} {pct:5.1f}% "
                    f"{rec/MB:7.2f}/{total/MB:.2f} MB | {speed:5.1f} MB/s")
                sys.stdout.flush()
    print(f"\n✅ Downloaded → {path}")
    return path, total

# ───────────────────────────────────────── WORKFLOW
headers = {'Authorization': f'Bearer {API_KEY}'} if API_KEY else {}
size_bytes = int(requests.head(MODEL_URL, allow_redirects=True, headers=headers)
                 .headers.get('Content-Length', 0))
size_gb = size_bytes/1024/1024/1024

if size_gb > 15:
    print(f"⚠️ File is {size_gb:.1f} GB (>15 GB). Using split workflow.")
    work_dir = '/content/large_model_temp'; os.makedirs(work_dir, exist_ok=True)
    big_file, _ = download_with_progress(MODEL_URL, work_dir, API_KEY)

    print("🔧 Splitting into 12 GB chunks…")
    subprocess.run(['apt','-y','install','zip'], check=False)
    zip_base = 'WanAI_split.zip'
    subprocess.run(['zip','-s','12000m',zip_base, os.path.basename(big_file)],
                   cwd=work_dir, check=True)

    parts = sorted(glob.glob(os.path.join(work_dir, zip_base+'*')))
    targets = ['/content/drive/MyDrive/WanAI',
               '/content/drive/MyDrive/WanAI2',
               '/content/drive/MyDrive/WanAI3']
    for t in targets: os.makedirs(t, exist_ok=True)

    for idx, part in enumerate(parts):
        tgt = targets[idx % len(targets)]
        print(f"📤 Copying {os.path.basename(part)} → {tgt}")
        shutil.copy(part, tgt)

    print("🎉 All chunks distributed across WanAI drives.")
else:
    print(f"File is {size_gb:.1f} GB — normal download.")
    save_to = MODEL_DIRS.get(MODEL_TYPE, os.path.join(BASE_DIR,'models'))
    download_with_progress(MODEL_URL, save_to, API_KEY)


🔗 Mounting Google Drive…
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚠️ File is 30.5 GB (>15 GB). Using split workflow.
📦 File size: 31272.29 MB


/tmp/ipython-input-1-1595485322.py:43: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi; _, prm = cgi.parse_header(r.headers['content-disposition'])


📥 wan2.1_i2v_480p_14B_fp16.safetensors [████--------------------------]  14.9% 4668.17/31272.29 MB |   9.2 MB/s

In [ ]:
#@title 🔧 ComfyUI Launcher (archive‑auto‑extract, pip‑cache)
# ───────────────────────────── setup options
storage_location = "Google Drive"  #@param ["Google Drive", "Colab Runtime", "SDUI Temp Folder"]
archive_folders  = "drive/MyDrive/WanAI,drive/MyDrive/WanAI2,drive/MyDrive/WanAI3"  #@param {type:"string"}
show_logs        = True  #@param {type:"boolean"}
custom_flags     = "--highvram --dont-print-server" #@param {type:"string"}
required_modules = ['torch','flask','flask_cors','requests','torchsde','spandrel','kornia','piexif','segment_anything'] #@param {type:"string"}

import os, subprocess, threading, time, socket, glob, shutil, re
from google.colab import drive

# ───────────────────────────── mount & paths
if storage_location == "Google Drive":
    print("🔗 Mounting Drive…")
    drive.mount('/content/drive')
    folder_path = "/content/drive/MyDrive/ComfyUI"
elif storage_location == "SDUI Temp Folder":
    folder_path = "/content/sdui/ComfyUI"
else:
    folder_path = "/content/ComfyUI"
os.makedirs(folder_path, exist_ok=True)
print("📁 Using:", folder_path)

# ───────────────────────────── pip cache (persistent wheels)
PIP_CACHE = "/content/drive/MyDrive/.pip_cache"
os.makedirs(PIP_CACHE, exist_ok=True)
os.environ["PIP_CACHE_DIR"] = PIP_CACHE

def install_module(m):
    try:
        __import__(m)
    except ModuleNotFoundError:
        print("📦 Installing", m)
        subprocess.run(["pip","install","-q",m])

for mod in required_modules: install_module(mod)

# ───────────────────────────── archive‑extract worker
COMFY_MODELS = os.path.join(folder_path, "models")
os.makedirs(COMFY_MODELS, exist_ok=True)

def extract_archives():
    print("🗃️  Archive extractor thread started…")
    any_found=False
    for root in ["/content/"+p.strip() if not p.startswith("/") else "/"+p.strip()
                 for p in archive_folders.split(",") if p.strip()]:
        parts = glob.glob(os.path.join(root, "*.zip.001"))
        for p001 in parts:
            any_found=True
            base = re.sub(r'\.zip\.001$', '', os.path.basename(p001))  # e.g. diffusion_models_wanai
            model_type = base.split("_")[0]                             # diffusion_models
            dest_dir   = os.path.join(COMFY_MODELS, model_type)
            os.makedirs(dest_dir,  exist_ok=True)

            tmp = "/content/tmp_zip"; shutil.rmtree(tmp, ignore_errors=True); os.makedirs(tmp)
            for part in glob.glob(os.path.join(root, base+".zip*")): shutil.copy(part, tmp)

            print(f"🔧 Extracting {base} → {model_type}")
            subprocess.run(["7z","x",f"{base}.zip",f"-o{dest_dir}","-y"], cwd=tmp)
            shutil.rmtree(tmp, ignore_errors=True)
            print("✅ Done:", base)
    if not any_found:
        print("ℹ️  No split archives detected.")

threading.Thread(target=extract_archives, daemon=True).start()

# ───────────────────────────── Cloudflared (same as before)
if not os.path.exists("/usr/bin/cloudflared"):
    subprocess.run(["wget","-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb","-O","/tmp/cf.deb"])
    subprocess.run(["dpkg","-i","/tmp/cf.deb"])

def tunnel(port):
    import socket, time
    while True:
        try: socket.create_connection(("127.0.0.1", port),1); break
        except: time.sleep(0.5)
    p = subprocess.Popen(["cloudflared","tunnel","--url",f"http://127.0.0.1:{port}"],stderr=subprocess.PIPE)
    for line in p.stderr:
        txt=line.decode()
        if "trycloudflare.com" in txt:
            print("🌐 Public URL:", txt[txt.find("http"):], end='')

threading.Thread(target=tunnel, daemon=True, args=(8188,)).start()

# ───────────────────────────── launch ComfyUI
print("🚀 Booting ComfyUI…")
main_py = os.path.join(folder_path, "main.py")
if show_logs:
    os.system(f"python {main_py} {custom_flags}")
else:
    subprocess.Popen(["python", main_py, *custom_flags.split()], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    print("✅ Running silently. Set 'show_logs' to True to watch output.")


🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Using storage at: /content/drive/MyDrive/ComfyUI
🚀 CUDA Available!


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#@title **📦 Compress & Clean Output Folder**
#@markdown Use this tool to compress your output folder and clean it after.
#@markdown Choose file format, compression level, and destination.
!pip install py7zr
!pip install rarfile
import os
import shutil
import zipfile
import sys
import subprocess
import py7zr
import rarfile
from tqdm import tqdm
from google.colab import drive

# ✅ **Mount Google Drive**
mount_gdrive = True  #@param {type:"boolean"}
if mount_gdrive:
    drive.mount('/content/drive')

# ✅ **User Options**
output_folder = "/content/drive/MyDrive/ComfyUI/output"  #@param {type: "string"}
archive_name = "output"  #@param {type: "string"}
archive_destination = "/content/drive/MyDrive/"  #@param {type: "string"}

file_type = "zip"  #@param ["zip", "7z", "rar"]
compression_level = "normal"  #@param ["store", "fast", "normal", "high", "maximum"]

# 🔥 **Compression Level Mapping**
compression_map = {
    'store': 0, 'fast': 1, 'normal': 5, 'high': 7, 'maximum': 9
}

def compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level):
    """Compresses and cleans the output folder."""

    if not os.path.exists(output_folder):
        print(f"❌ Error: The folder '{output_folder}' does not exist.")
        return

    if not os.path.exists(archive_destination):
        print(f"❌ Error: The destination folder '{archive_destination}' does not exist.")
        return

    total_original_size = sum(
        os.path.getsize(os.path.join(root, file))
        for root, _, files in os.walk(output_folder)
        for file in files
    )

    extension = {'zip': '.zip', '7z': '.7z', 'rar': '.rar'}[file_type]
    archive_path = os.path.join(archive_destination, f"{archive_name}{extension}")
    compression_value = compression_map[compression_level]

    # ✅ **ZIP Compression**
    if file_type == 'zip':
        compression = zipfile.ZIP_STORED if compression_level == 'store' else zipfile.ZIP_DEFLATED
        with zipfile.ZipFile(archive_path, 'w', compression) as zipf:
            for root, _, files in os.walk(output_folder):
                for file in tqdm(files, desc="📦 Compressing as .zip", unit="file"):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_folder))

    # ✅ **7Z Compression**
    elif file_type == '7z':
        with py7zr.SevenZipFile(archive_path, 'w', filters=[{'id': py7zr.FILTER_LZMA2, 'preset': compression_value}]) as archive:
            for root, _, files in os.walk(output_folder):
                for file in tqdm(files, desc="📦 Compressing as .7z", unit="file"):
                    archive.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_folder))

    # ✅ **RAR Compression**
    elif file_type == 'rar':
        try:
            print("📦 Compressing as .rar (This may take some time, progress bar not supported)")
            subprocess.run(['rar', 'a', '-m' + str(compression_value), archive_path, output_folder], check=True)
        except subprocess.CalledProcessError:
            print("❌ Failed to create .rar archive. Install 'rar' using 'apt-get install rar'.")

    total_compressed_size = os.path.getsize(archive_path)
    compression_percentage = (1 - (total_compressed_size / total_original_size)) * 100 if total_original_size > 0 else 0

    print(f"📦 Original size: {total_original_size / 1024**2:.2f} MB")
    print(f"📦 Compressed size: {total_compressed_size / 1024**2:.2f} MB")
    print(f"📦 Compression percentage: {compression_percentage:.2f}%")

    # ✅ **Delete original files**
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'❌ Failed to delete {file_path}. Reason: {e}')

    print(f"✅ Successfully compressed and cleaned {output_folder}")

# 🔥 **Run the function**
compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


📦 Compressing as .zip: 100%|██████████| 70/70 [00:18<00:00,  3.85file/s]


📦 Original size: 336.22 MB
📦 Compressed size: 335.62 MB
📦 Compression percentage: 0.18%
✅ Successfully compressed and cleaned /content/drive/MyDrive/ComfyUI/output
